reformat metadata for https://www.youtube.com/watch?v=i0b29lAuMlg&feature=youtu.be

In [1]:
from classical_music import *

file_txt = "100-classical-pieces.txt"

file_html = file_txt.replace(".txt", ".html")

In [2]:
meta_map, lines = parse_file_txt(file_txt)

meta_map, lines[:2]

({'vid': 'K3He_GSvFzY',
  'vid_name': '100 Classical Pieces',
  'channel_id': 'UCyOfqgtsQaM3S-VZnsYnHjQ',
  'channel_name': 'HALIDONMUSIC'},
 ['1-3 Vivaldi - The Four Seasons, ‘Spring’ 00:00',
  '4-7 Telemann - Viola Concerto TWV 09:00'])

In [3]:
# this is data file specific
def parse_line(l):
    """parse text-line into data elements
    
    sample:
        11-13 Bach - Harpsichord Concerto No. 1 (Arr. for Piano) I. Allegro 28:19 
        
    returns:
        [id, ts, name, title]
    """
    tmp = l.split(" - ")
    tmp1 = tmp[0].split(" ")
    _id, last_name = tmp1[0], " ".join(tmp1[1:])
    tmp2 = [i.strip() for i in tmp[-1].split(" ") if i.strip()]
    title = " ".join(tmp2[:-1])
    ts = tmp2[-1]
    return [_id.strip(), ts.strip(), last_name.strip(), title.strip()]

In [4]:
line = "11-13 Bach - Harpsichord Concerto No. 1 (Arr. for Piano) I. Allegro 28:19 "
d = parse_line(line)
d

['11-13',
 '28:19',
 'Bach',
 'Harpsichord Concerto No. 1 (Arr. for Piano) I. Allegro']

In [5]:
records = []
for i in lines:
    i = i.strip()
    if not i: 
        continue
    d = parse_line(i)
    # records.append(d)
    if d[2] not in name_map:
        print(f"[ERROR] {d[2]} not found in name_map")
    else:
        records.append([d[0].replace(".", ""), make_youtube_link(meta_map["vid"], d[1]), make_wikiwand_link(name_map[d[2]]), d[-1]])

[ERROR] Floridia not found in name_map
[ERROR] Einaudi not found in name_map
[ERROR] Rota not found in name_map
[ERROR] Lanzetta not found in name_map
[ERROR] Rota not found in name_map


In [6]:
records[:3]

[['1-3',
  '<a href=https://www.youtube.com/watch?v=K3He_GSvFzY&t=0s>00:00:00</a>',
  '<a href=https://www.wikiwand.com/en/Antonio_Vivaldi>Antonio Vivaldi</a>',
  'The Four Seasons, ‘Spring’'],
 ['4-7',
  '<a href=https://www.youtube.com/watch?v=K3He_GSvFzY&t=540s>00:09:00</a>',
  '<a href=https://www.wikiwand.com/en/Georg_Philipp_Telemann>Georg Philipp Telemann</a>',
  'Viola Concerto TWV'],
 ['8-10',
  '<a href=https://www.youtube.com/watch?v=K3He_GSvFzY&t=1139s>00:18:59</a>',
  '<a href=https://www.wikiwand.com/en/Antonio_Vivaldi>Antonio Vivaldi</a>',
  'Symphony RV. 112']]

In [7]:
# write out HTML file
with open(file_html, "w") as f:
    
    f.write(f"""<h1>
    <a href=https://www.youtube.com/watch?v={meta_map["vid"]}>{meta_map["vid_name"]}</a> </h1> 
    <h2>by <a href=https://www.youtube.com/channel/{meta_map["channel_id"]}>{meta_map["channel_name"]}</a>
    </h2>
    <br>
    <table style="width:100%" border=1px>
    """)
    
    f.write("""<tr>
        <th>Id</th>
        <th>Track TS</th>
        <th>Musician</th>
        <th>Title</th>
      </tr>""")
    for r in records:
        f.write(f"""<tr>
            <td>{r[0]}</td>
            <td>{r[1]}</td>
            <td>{r[2]}</td>
            <td>{r[3]}</td>
          </tr>""")
    f.write('</table>')